In [ ]:

import os
import cv2
import numpy as np
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Concatenate
from tensorflow.keras import regularizers
import tensorflow as tf


In [ ]:
# Set up image size

WIDTH, HEIGHT = 320, 240
HEIGHT_REQUIRED_PORTION = 0.5
WIDTH_REQUIRED_PORTION = 0.9

height_from = int(HEIGHT * (1 - HEIGHT_REQUIRED_PORTION))
width_from = int((WIDTH - WIDTH * WIDTH_REQUIRED_PORTION) / 2)
width_to = width_from + int(WIDTH_REQUIRED_PORTION * WIDTH)

new_height = HEIGHT - height_from
new_width = width_to - width_from
image_size = (new_width, new_height)

MAX_STEER_DEGREES = 40


In [ ]:

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (WIDTH, HEIGHT))
    image = image[height_from:, width_from:width_to]
    return image / 255.0


In [ ]:
# Create a custom data generator

def custom_data_generator(image_files, batch_size=64):
    while True:
        batch = random.sample(image_files, batch_size)
        batch_images, batch_input_2, batch_labels = [], [], []
        for path in batch:
            parts = os.path.basename(path).split('_')
            steer = float(parts[2].replace('.png', ''))
            steer = np.clip(steer, -MAX_STEER_DEGREES, MAX_STEER_DEGREES) / MAX_STEER_DEGREES
            input_2 = int(parts[1])
            image = preprocess_image(path)
            batch_images.append(image)
            batch_input_2.append(input_2)
            batch_labels.append(steer)
        yield [np.array(batch_images), np.array(batch_input_2)], np.array(batch_labels)


In [ ]:
# Create a CNN model to extract features

def create_feature_extractor():
    image_input = Input(shape=(new_height, new_width, 3))
    int_input = Input(shape=(1,))

    x = Conv2D(64, (6, 6), activation='relu', padding='same')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (6, 6), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (6, 6), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (6, 6), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Dense(8, activation='relu', name='feature_dense')(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation='relu')(x)
    x = Flatten()(x)

    features = Concatenate()([x, int_input])
    output = Dense(1)(features)  # steer 值输出，用于训练

    model = Model(inputs=[image_input, int_input], outputs=output)
    return model


In [ ]:
# Get a list of image file paths and labels

data_dir = 'D:\FP\Carla\CNN/_img'
image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.png')]
random.shuffle(image_files)
split = int(0.8 * len(image_files))
train_files, val_files = image_files[:split], image_files[split:]

train_gen = custom_data_generator(train_files, batch_size=64)
val_gen = custom_data_generator(val_files, batch_size=64)

model = create_feature_extractor()
model.compile(optimizer='adam', loss='mse')
model.summary()

model.fit(
    train_gen,
    steps_per_epoch=len(train_files)//64,
    validation_data=val_gen,
    validation_steps=len(val_files)//64,
    epochs=10
)


In [ ]:
# Save dense layer output

feature_output = model.get_layer('feature_dense').output
feature_model = Model(inputs=model.input, outputs=feature_output)
feature_model.save("model_saved_from_CNN.h5")
